In [1]:
import pandas as pd
import numpy as np

In [2]:
HofP=pd.read_json('./HeroesOfPymoli/purchase_data.json')
HofP.head(10)

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


### Player Count

In [3]:
pd.DataFrame(data={'Total Players': [HofP['SN'].nunique()]})

,Total Players
0,573


### Purchasing Analysis (Total)

In [4]:
keys =['Number of Unique Items','Average Price','Number of Purchases','Total Revenue']
unique_items= HofP['Item ID'].nunique()                  
ave_price = HofP['Price'].mean()
total_purchase = HofP['Item ID'].count()
total_revenue = HofP['Price'].sum()
purchase_ana = {'Number of Unique Items': [unique_items],'Average Price': [ave_price],
                'Number of Purchases': [total_purchase],'Total Revenue': [total_revenue]}
purchase_ana_df = pd.DataFrame(data=purchase_ana, columns=keys)
purchase_ana_df['Average Price'] = purchase_ana_df['Average Price'].map('${:,.2f}'.format)
purchase_ana_df['Total Revenue'] = purchase_ana_df['Total Revenue'].map('${:,.2f}'.format)
purchase_ana_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


### Gender Demographics

In [5]:
cols=['Percentage of Players',' Total Count ']
genders=[HofP['Gender'].unique()]
gender_grp_counts=pd.DataFrame(HofP.groupby(HofP['Gender'])['SN'].nunique(), 
                               index=genders)
per_plyr=pd.DataFrame(gender_grp_counts['SN']/gender_grp_counts['SN'].sum()*100)
res=per_plyr.set_index(genders).join(gender_grp_counts.set_index(genders),
                                                     lsuffix='0', rsuffix='1')
res['SN0'] = res['SN0'].map('{:,.2f}'.format)
res.rename(columns={'SN0':cols[0],'SN1':cols[1]})

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


### Purchasing Analysis (Gender)

In [6]:
HofP['Total Purchase Value']=HofP['Price']
HofP['Normalized Totals']=pd.DataFrame(HofP.groupby(['Gender'])['Price']
                                       .apply(lambda x: (x) / x.count()))
pur_ana=pd.DataFrame(HofP.groupby(['Gender']).agg({'Item ID': 'count',
                                                    'Price': 'mean',
                                                   'Normalized Totals':'sum',
                                                     })).rename(columns={'Item ID':'Purchase Count',
                                                                         'Price':'Average Price'})
pur_ana['Total Purchase Value']= pur_ana['Purchase Count']*pur_ana['Average Price']
pur_ana['Total Purchase Value']= pur_ana['Total Purchase Value'].map('${:,.2f}'.format)
pur_ana['Normalized Totals']= pur_ana['Normalized Totals'].map('${:,.2f}'.format)
pur_ana['Average Price']= pur_ana['Average Price'].map('${:,.2f}'.format)
pur_ana

,Purchase Count,Average Price,Normalized Totals,Total Purchase Value
Gender,,,,
Female,136,$2.82,$2.82,$382.91
Male,633,$2.95,$2.95,"$1,867.68"
Other / Non-Disclosed,11,$3.25,$3.25,$35.74


In [7]:
HofP1=HofP
HofP1['Percent Count']=HofP['SN']
HofP1['Total Count']=HofP['SN']

def Pyr_count(SN):
    return (len(SN.unique()))
def Perc_Pyr(SN):
    return (len(SN.unique())/HofP['SN'].nunique()*100)

rlabel=[[ "{0}-{1}".format(i+1, i + 4) for i in range(HofP['Age'].min()-1,
                                                       HofP['Age'].max()+4, 4) ]]
rlabel[0][0]='<'+ rlabel[0][0][-2:]
rlabel[0][len(rlabel[0])-1]=rlabel[0][len(rlabel[0])-1][:2]+'+'

a=HofP1.groupby(pd.cut(HofP1['Age'] , 
                    bins=np.arange(HofP1['Age'].min()-1,
                                   HofP1['Age'].max()+8,4),
                      right=True)).agg({'Percent Count':Perc_Pyr,
                                       'Total Count':Pyr_count}
                                      )
a.index=rlabel
a.fillna(0, inplace=True)
a['Total Count']=a['Total Count'].astype(int)
a['Percent Count']=a['Percent Count'].map('{:,.2f}'.format)
a

,Percent Count,Total Count
<10,3.84,22
11-14,3.49,20
15-18,14.66,84
19-22,31.06,178
23-26,26.70,153
27-30,7.68,44
31-34,5.93,34
35-38,4.36,25
39-42,1.92,11
43-46,0.35,2


### Purchasing Analysis (Age)

In [9]:
def norm(price):
    price_list=list(price)
    norm=map(lambda x: (x-np.min(price_list)) / (np.max(price_list)-np.min(price_list)),price)
    norm=(sum(list(norm)))
    return norm
def total_pur_val(price):
    return np.mean(price)*len(price)

HofP1['Normalized Totals']=HofP1['Price']
a=HofP1.groupby(pd.cut(HofP1['Age'] , 
                    bins=np.arange(HofP1['Age'].min()-1,
                                   HofP1['Age'].max()+8,4),
                      right=True)).agg({'Item ID':'count',
                                       'Price':'mean',
                                        'Total Purchase Value': total_pur_val,
                                       'Normalized Totals':norm}
                                      ).rename(columns={'Item ID':'Purchase Count',
                                                       'Price':'Average Purchase Price'})

a['Average Purchase Price']=a['Average Purchase Price'].map('${:,.2f}'.format)
a['Total Purchase Value']=a['Total Purchase Value'].map('${:,.2f}'.format)
a['Normalized Totals']=a['Normalized Totals'].map('${:,.2f}'.format)
a.index=rlabel
a.sort_index(0)

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
11-14,31,$2.70,$83.79,$12.92
15-18,111,$2.88,$319.32,$52.29
19-22,231,$2.93,$676.20,$111.80
23-26,207,$2.94,$608.02,$100.72
27-30,63,$2.98,$187.99,$31.40
31-34,46,$3.07,$141.24,$23.94
35-38,37,$2.81,$104.06,$18.37
39-42,20,$3.13,$62.56,$11.19
43-46,2,$3.26,$6.53,$1.00
47+,0,$nan,$nan,$nan


### Top Spenders

In [10]:
HofP1['Normalized Totals']=HofP1['Price']
Tp_Spendr=pd.DataFrame(HofP1.groupby(['SN']).agg({'Item ID': 'count',
                                                    'Price': 'mean',
                                                   'Total Purchase Value':total_pur_val,
                                                     })).rename(columns={'Item ID':'Purchase Count',
                                                                         'Price':'Average Purchase Price'})
Tp_Spendr['Average Purchase Price']=Tp_Spendr['Average Purchase Price'].map('${:,.2f}'.format)
Tp_Spendr['Total Purchase Value']=Tp_Spendr['Total Purchase Value'].map('${:,.2f}'.format)
Tp_Spendr=Tp_Spendr.sort_values(by=['Purchase Count'], ascending=False)
Tp_Spendr.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Mindimnya67,4,$3.18,$12.74
Qarwen67,4,$2.49,$9.97
Saedue76,4,$3.39,$13.56
Sondastan54,4,$2.56,$10.24


### Most Popular Items

In [11]:
Tp_items=pd.DataFrame(HofP1.groupby(['Item ID', 'Item Name']).agg({'Item ID': 'count',
                                                    'Price': 'mean',
                                                   'Total Purchase Value':total_pur_val,
                                                     })).rename(columns={'Item ID':'Purchase Count',
                                                                         'Price':'Item Price'})
Tp_items['Item Price']=Tp_items['Item Price'].map('${:,.2f}'.format)
Tp_items['Total Purchase Value']=Tp_items['Total Purchase Value'].map('${:,.2f}'.format)
Tp_items=Tp_items.sort_values(by=['Purchase Count'], ascending=False)
Tp_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


### Profitable Items

In [12]:
Tp_items=Tp_items.sort_values(by=['Total Purchase Value'], ascending=False)
Tp_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
170,Shadowsteel,5,$1.98,$9.90
21,Souleater,3,$3.27,$9.81
37,"Shadow Strike, Glory of Ending Hope",5,$1.93,$9.65
127,"Heartseeker, Reaver of Souls",3,$3.21,$9.63
120,Agatha,5,$1.91,$9.55
